In [13]:
import pandas as pd
import glob
import os

In [49]:
path = "/Users/pranav/Documents/development/njtransit/csv/2018_09"

In [50]:
allFiles = glob.glob(os.path.join(path, "*.csv"))

In [51]:
df = pd.concat((pd.read_csv(f) for f in sorted(allFiles)))

In [52]:
df.to_csv("/Users/pranav/Documents/development/njtransit/csv/monthly/2018_09.csv")